#**Aula 5: Criando um Chatbot**

##**Selecionando o modelo**

Modelo: [microsoft/DialoGPT-medium](https://huggingface.co/microsoft/DialoGPT-medium)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Hello!
DialoGPT: Hello! :D
>> User:How are you doing?
DialoGPT: I'm doing well! How are you?
>> User:I'm fine
DialoGPT: That's good!
>> User:Have a good day
DialoGPT: You too! :D
>> User:bye
DialoGPT: Bye! :D


##**Criando o chatbot**

In [ ]:
import pandas as pd

In [ ]:
dados_pedidos = {
    "numero_pedido": ["12345", "67890", "11121", "22232"],
    "status": ["Shipped", "Processing", "Delivered", "Cancelled"]
}
df_status_pedidos = pd.DataFrame(dados_pedidos)

In [ ]:
df_status_pedidos

,numero_pedido,status
0,12345,Shipped
1,67890,Processing
2,11121,Delivered
3,22232,Cancelled


In [ ]:
def verificar_status_pedido(numero_pedido):
  try:
    status = df_status_pedidos[df_status_pedidos['numero_pedido'] == numero_pedido]['status'].iloc[0]

    return f'The status of your order {numero_pedido} is: {status}'

  except:
    return 'Order number not found. Please check and try again'

In [ ]:
palavras_chave_status = ['order', 'order status', 'status of my order', 'check my order', 'track my order', 'order update']

##**Interagindo com o chatbot**

In [ ]:
ids_historico_chat = None

while True:
  input_usuario = input('You: ')

  if input_usuario.lower() in ['exit', 'quit', 'stop']:
    print('Bot: Goodbye!')
    break

  if any(keyword in input_usuario.lower() for keyword in palavras_chave_status):
    numero_pedido = input('Could you please enter you order number?')
    resposta = verificar_status_pedido(numero_pedido)
  else:
    novo_usuario_input_ids = tokenizer.encode(input_usuario + tokenizer.eos_token, return_tensors='pt')

    if ids_historico_chat is not None:
      bot_input_ids = torch.cat([ids_historico_chat, novo_usuario_input_ids ], dim=-1)

    else:
      bot_input_ids = novo_usuario_input_ids

    ids_historico_chat = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id
    )
    resposta = tokenizer.decode(ids_historico_chat[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

  print(f'Bot: {resposta}')

You: Hello!!
Bot: Hello! :D
You: How are you doing?
Bot: I'm doing well! How are you?
You: I'm great
Bot: That's good!
You: I would like to check the status of my order
Could you please enter you order number?12345
Bot: The status of your order 12345 is: Shipped
You: Thanks a lot
Bot: No problem :D
You: stop
Bot: Goodbye!
